# ES Order Move to BBF BANs - Dry Run Analysis

**PREREQUISITE:** v8 BAN creation must be completed first (new -BBF BANs must exist)

## Purpose
This notebook analyzes what would happen when we move orders to new BBF BANs.
It does NOT make any changes - dry run only.

## What Gets Moved
- Orders WITHOUT "PA MARKET DECOM" in Project_Group__c
- That are on a BAN which has a new -BBF counterpart

## Updates Required (when we go live)
1. **SF:** Update `Order.Billing_Invoice__c` → new -BBF BAN SFID
2. **OSS:** Update `orders.account_id` → new OSS account_id

## Output
- Orders to move with old/new BAN mapping
- OSS order mapping with old/new account_id
- Data quality issues (missing OSS_Order__c, etc.)
- Summary stats

In [ ]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime

# Active statuses
ACTIVE_STATUSES = ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']

# WD identifier - orders with this stay on original BAN
WD_PROJECT_GROUP = 'PA MARKET DECOM'

# Connection credentials
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

print("Connecting to Heroku...")
conn = psycopg2.connect(**heroku_conn_kwargs)
print("✅ Connected to Heroku")

print("Connecting to OSS...")
oconn = psycopg2.connect(**oss_conn_kwargs)
print("✅ Connected to OSS")

In [ ]:
# === CONFIGURATION ===
OUTPUT_DIR = "."
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{OUTPUT_DIR}/es_order_move_dry_run_{timestamp}.xlsx"

print(f"Output file: {output_file}")

In [ ]:
# === STEP 1: Get all new -BBF BANs (created by v8) ===

bbf_bans_sql = """
SELECT 
    sfid AS new_ban_sfid,
    name AS new_ban_name,
    account_number__c AS new_account_id,
    legacy_es_id__c AS old_ban_sfid,
    account__c AS customer_sfid,
    account_name__c,
    bbf_ban__c
FROM sfprod.billing_invoice__c
WHERE legacy_es_id__c IS NOT NULL
  AND bbf_ban__c = true
ORDER BY name;
"""

print("Querying new -BBF BANs...")
bbf_bans_df = pd.read_sql_query(bbf_bans_sql, conn)
print(f"✅ Found {len(bbf_bans_df)} new -BBF BANs")

if len(bbf_bans_df) == 0:
    print("\n❌ ERROR: No -BBF BANs found!")
    print("   v8 BAN creation must be run first before this notebook.")
else:
    print(f"\nSample -BBF BANs:")
    print(bbf_bans_df[['new_ban_name', 'new_account_id', 'old_ban_sfid', 'account_name__c']].head(10))

# Build mapping dictionaries
old_to_new_ban_sfid = dict(zip(bbf_bans_df['old_ban_sfid'], bbf_bans_df['new_ban_sfid']))
old_to_new_ban_name = dict(zip(bbf_bans_df['old_ban_sfid'], bbf_bans_df['new_ban_name']))
old_to_new_account_id = dict(zip(bbf_bans_df['old_ban_sfid'], bbf_bans_df['new_account_id']))

print(f"\nBuilt mapping for {len(old_to_new_ban_sfid)} BANs")

In [ ]:
# === STEP 2: Get old BAN info for the mapping ===

old_ban_sfids = list(bbf_bans_df['old_ban_sfid'].dropna())
old_ban_sfids_str = "','".join(old_ban_sfids)

old_bans_sql = f"""
SELECT 
    sfid AS old_ban_sfid,
    name AS old_ban_name,
    account_number__c AS old_account_id
FROM sfprod.billing_invoice__c
WHERE sfid IN ('{old_ban_sfids_str}')
"""

print("Querying old BAN info...")
old_bans_df = pd.read_sql_query(old_bans_sql, conn)
print(f"✅ Found {len(old_bans_df)} old BANs")

# Add old BAN info to mapping
old_ban_sfid_to_name = dict(zip(old_bans_df['old_ban_sfid'], old_bans_df['old_ban_name']))
old_ban_sfid_to_account_id = dict(zip(old_bans_df['old_ban_sfid'], old_bans_df['old_account_id']))

In [ ]:
# === STEP 3: Get all orders that should move to -BBF BANs ===

status_str = "','".join(ACTIVE_STATUSES)

# Orders to move:
# - Active status
# - NOT PA MARKET DECOM
# - On a BAN that has a new -BBF counterpart

orders_to_move_sql = f"""
SELECT 
    o.sfid AS order_sfid,
    o.ordernumber,
    o.name AS order_name,
    o.service_id__c,
    o.status,
    o.billing_invoice__c AS current_ban_sfid,
    bi.name AS current_ban_name,
    bi.account_number__c AS current_account_id,
    o.oss_order__c,
    o.project_group__c,
    o.dimension_4_market__c AS market,
    o.accountid,
    a.name AS account_name,
    o.service_start_date__c,
    o.sof_mrc__c
FROM sfprod."order" o
JOIN sfprod.billing_invoice__c bi ON o.billing_invoice__c = bi.sfid
LEFT JOIN sfprod.account a ON o.accountid = a.sfid
WHERE o.status IN ('{status_str}')
  AND (o.project_group__c IS NULL OR o.project_group__c NOT LIKE '%{WD_PROJECT_GROUP}%')
  AND o.billing_invoice__c IS NOT NULL
ORDER BY bi.name, o.ordernumber;
"""

print("Querying orders to move (no PA MARKET DECOM)...")
all_bbf_orders_df = pd.read_sql_query(orders_to_move_sql, conn)
print(f"✅ Found {len(all_bbf_orders_df):,} total BBF orders (no PA MARKET DECOM)")

# Filter to only orders on BANs that have a -BBF counterpart
orders_to_move_df = all_bbf_orders_df[
    all_bbf_orders_df['current_ban_sfid'].isin(old_to_new_ban_sfid.keys())
].copy()

print(f"✅ {len(orders_to_move_df):,} orders are on BANs with -BBF counterpart (will move)")

# Orders on BANs without -BBF counterpart
orders_no_bbf_ban = all_bbf_orders_df[
    ~all_bbf_orders_df['current_ban_sfid'].isin(old_to_new_ban_sfid.keys())
].copy()

print(f"⚠️ {len(orders_no_bbf_ban):,} orders are on BANs WITHOUT -BBF counterpart")

In [ ]:
# === STEP 4: Add new BAN info to orders ===

orders_to_move_df['new_ban_sfid'] = orders_to_move_df['current_ban_sfid'].map(old_to_new_ban_sfid)
orders_to_move_df['new_ban_name'] = orders_to_move_df['current_ban_sfid'].map(old_to_new_ban_name)
orders_to_move_df['new_account_id'] = orders_to_move_df['current_ban_sfid'].map(old_to_new_account_id)

print("Added new BAN mapping to orders")
print(f"\nSample orders to move:")
print(orders_to_move_df[['ordernumber', 'current_ban_name', 'new_ban_name', 'current_account_id', 'new_account_id']].head(10))

In [ ]:
# === STEP 5: Check OSS Order linkage ===

# Orders with OSS_Order__c
orders_with_oss = orders_to_move_df[orders_to_move_df['oss_order__c'].notna()].copy()
orders_without_oss = orders_to_move_df[orders_to_move_df['oss_order__c'].isna()].copy()

print(f"\n=== OSS Order Linkage ===")
print(f"✅ Orders WITH OSS_Order__c: {len(orders_with_oss):,}")
print(f"❌ Orders WITHOUT OSS_Order__c: {len(orders_without_oss):,}")

if len(orders_without_oss) > 0:
    print(f"\n⚠️ WARNING: {len(orders_without_oss):,} orders cannot be updated in OSS (no OSS_Order__c link)")
    print("\nSample orders missing OSS_Order__c:")
    print(orders_without_oss[['ordernumber', 'service_id__c', 'current_ban_name', 'status']].head(10))

In [ ]:
# === STEP 6: Verify OSS orders exist ===

oss_order_ids = orders_with_oss['oss_order__c'].dropna().astype(int).tolist()

if len(oss_order_ids) > 0:
    # Query in batches to avoid query size limits
    batch_size = 5000
    oss_orders_list = []
    
    print(f"\nVerifying {len(oss_order_ids):,} OSS orders exist...")
    
    for i in range(0, len(oss_order_ids), batch_size):
        batch = oss_order_ids[i:i+batch_size]
        batch_str = ','.join(map(str, batch))
        
        oss_orders_sql = f"""
        SELECT 
            order_id,
            account_id AS current_oss_account_id,
            order_status
        FROM orders.orders
        WHERE order_id IN ({batch_str})
        """
        
        batch_df = pd.read_sql_query(oss_orders_sql, oconn)
        oss_orders_list.append(batch_df)
        print(f"  Batch {i//batch_size + 1}: {len(batch_df)} found")
    
    oss_orders_df = pd.concat(oss_orders_list, ignore_index=True)
    print(f"\n✅ Found {len(oss_orders_df):,} OSS orders")
    
    # Check for missing OSS orders
    found_oss_ids = set(oss_orders_df['order_id'].tolist())
    missing_oss_ids = set(oss_order_ids) - found_oss_ids
    
    if missing_oss_ids:
        print(f"\n❌ WARNING: {len(missing_oss_ids)} OSS_Order__c values do not exist in OSS!")
    else:
        print(f"\n✅ All OSS_Order__c values found in OSS")
else:
    print("No OSS orders to verify")
    oss_orders_df = pd.DataFrame()

In [ ]:
# === STEP 7: Build complete order move mapping ===

# Merge OSS current account_id into orders_with_oss
if len(oss_orders_df) > 0:
    orders_with_oss['oss_order__c_int'] = orders_with_oss['oss_order__c'].astype(int)
    orders_with_oss = orders_with_oss.merge(
        oss_orders_df[['order_id', 'current_oss_account_id']],
        left_on='oss_order__c_int',
        right_on='order_id',
        how='left'
    )
    orders_with_oss.drop(columns=['order_id', 'oss_order__c_int'], inplace=True, errors='ignore')

print("\n=== Complete Order Move Mapping ===")
print(f"Orders with full mapping (SF + OSS): {len(orders_with_oss):,}")
print(f"\nSample complete mapping:")
print(orders_with_oss[['ordernumber', 'current_ban_name', 'new_ban_name', 
                       'current_account_id', 'new_account_id', 
                       'oss_order__c', 'current_oss_account_id']].head(10))

In [ ]:
# === STEP 8: Summary Statistics ===

print("\n" + "="*70)
print("SUMMARY")
print("="*70)

print(f"\n{'Category':<50} {'Count':>10}")
print("-" * 65)
print(f"{'New -BBF BANs created (v8)':<50} {len(bbf_bans_df):>10,}")
print(f"{'Total BBF orders (no PA MARKET DECOM)':<50} {len(all_bbf_orders_df):>10,}")
print(f"{'Orders on BANs WITH -BBF counterpart (will move)':<50} {len(orders_to_move_df):>10,}")
print(f"{'Orders on BANs WITHOUT -BBF counterpart':<50} {len(orders_no_bbf_ban):>10,}")
print()
print(f"{'Orders WITH OSS_Order__c (can update OSS)':<50} {len(orders_with_oss):>10,}")
print(f"{'Orders WITHOUT OSS_Order__c (SF only)':<50} {len(orders_without_oss):>10,}")

if len(oss_orders_df) > 0:
    print(f"{'OSS orders verified exist':<50} {len(oss_orders_df):>10,}")
    print(f"{'OSS orders NOT found (missing)':<50} {len(missing_oss_ids):>10,}")

print()
print(f"{'ORDERS TO MOVE (SF Billing_Invoice__c update)':<50} {len(orders_to_move_df):>10,}")
print(f"{'OSS ORDERS TO UPDATE (account_id update)':<50} {len(orders_with_oss):>10,}")

In [ ]:
# === STEP 9: Create Output Excel ===

wb = Workbook()

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill_blue = PatternFill("solid", fgColor="4472C4")
header_fill_green = PatternFill("solid", fgColor="70AD47")
header_fill_orange = PatternFill("solid", fgColor="ED7D31")
header_fill_red = PatternFill("solid", fgColor="C00000")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
thin_border = Border(
    left=Side(style='thin'), right=Side(style='thin'),
    top=Side(style='thin'), bottom=Side(style='thin')
)

def style_header(ws, headers, fill_color):
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = fill_color
        cell.alignment = header_alignment
        cell.border = thin_border

def auto_width(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column].width = min(max_length + 2, 50)

# === Sheet 1: Summary ===
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["ES Order Move to BBF BANs - Dry Run Analysis"])
ws1['A1'].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Generated:", datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
ws1.append([])
ws1.append(["PREREQUISITE: v8 BAN creation must be completed first"])
ws1['A5'].font = Font(bold=True, color="FF0000")
ws1.append([])
ws1.append(["BAN Mapping:"])
ws1['A7'].font = Font(bold=True)
ws1.append(["  New -BBF BANs created:", len(bbf_bans_df)])
ws1.append([])
ws1.append(["Order Counts:"])
ws1['A10'].font = Font(bold=True)
ws1.append(["  Total BBF orders (no PA MARKET DECOM):", len(all_bbf_orders_df)])
ws1.append(["  Orders on BANs WITH -BBF (will move):", len(orders_to_move_df)])
ws1.append(["  Orders on BANs WITHOUT -BBF:", len(orders_no_bbf_ban)])
ws1.append([])
ws1.append(["OSS Linkage:"])
ws1['A15'].font = Font(bold=True)
ws1.append(["  Orders WITH OSS_Order__c:", len(orders_with_oss)])
ws1.append(["  Orders WITHOUT OSS_Order__c:", len(orders_without_oss)])
if len(oss_orders_df) > 0:
    ws1.append(["  OSS orders verified exist:", len(oss_orders_df)])
    ws1.append(["  OSS orders NOT found:", len(missing_oss_ids)])
ws1.append([])
ws1.append(["ACTIONS REQUIRED:"])
ws1['A' + str(ws1.max_row)].font = Font(bold=True)
ws1.append(["  SF: Update Order.Billing_Invoice__c", len(orders_to_move_df)])
ws1.append(["  OSS: Update orders.account_id", len(orders_with_oss)])

# === Sheet 2: Orders to Move ===
ws2 = wb.create_sheet("Orders to Move")
headers2 = ['Order Number', 'Service ID', 'Status', 'Account Name', 
            'Current BAN Name', 'New BAN Name', 
            'Current Account ID', 'New Account ID',
            'OSS Order ID', 'Current OSS Account ID',
            'Market', 'MRC']
ws2.append(headers2)
style_header(ws2, headers2, header_fill_green)

for _, row in orders_with_oss.iterrows():
    ws2.append([
        row['ordernumber'],
        row['service_id__c'],
        row['status'],
        row['account_name'],
        row['current_ban_name'],
        row['new_ban_name'],
        row['current_account_id'],
        row['new_account_id'],
        row['oss_order__c'],
        row.get('current_oss_account_id', ''),
        row['market'],
        row['sof_mrc__c']
    ])

ws2.freeze_panes = 'A2'
auto_width(ws2)

# === Sheet 3: BAN Mapping ===
ws3 = wb.create_sheet("BAN Mapping")
headers3 = ['Old BAN SFID', 'Old BAN Name', 'Old Account ID',
            'New BAN SFID', 'New BAN Name', 'New Account ID', 'Account Name']
ws3.append(headers3)
style_header(ws3, headers3, header_fill_blue)

for _, row in bbf_bans_df.iterrows():
    ws3.append([
        row['old_ban_sfid'],
        old_ban_sfid_to_name.get(row['old_ban_sfid'], ''),
        old_ban_sfid_to_account_id.get(row['old_ban_sfid'], ''),
        row['new_ban_sfid'],
        row['new_ban_name'],
        row['new_account_id'],
        row['account_name__c']
    ])

ws3.freeze_panes = 'A2'
auto_width(ws3)

# === Sheet 4: Orders Missing OSS Link ===
ws4 = wb.create_sheet("Missing OSS Link")
headers4 = ['Order Number', 'Service ID', 'Status', 'Account Name',
            'Current BAN Name', 'New BAN Name', 'Market']
ws4.append(headers4)
style_header(ws4, headers4, header_fill_orange)

for _, row in orders_without_oss.iterrows():
    ws4.append([
        row['ordernumber'],
        row['service_id__c'],
        row['status'],
        row['account_name'],
        row['current_ban_name'],
        row['new_ban_name'],
        row['market']
    ])

ws4.freeze_panes = 'A2'
auto_width(ws4)

# === Sheet 5: Orders Without BBF BAN ===
ws5 = wb.create_sheet("No BBF BAN Available")
headers5 = ['Order Number', 'Service ID', 'Status', 'Account Name',
            'Current BAN Name', 'Current BAN SFID', 'Market']
ws5.append(headers5)
style_header(ws5, headers5, header_fill_red)

for _, row in orders_no_bbf_ban.iterrows():
    ws5.append([
        row['ordernumber'],
        row['service_id__c'],
        row['status'],
        row['account_name'],
        row['current_ban_name'],
        row['current_ban_sfid'],
        row['market']
    ])

ws5.freeze_panes = 'A2'
auto_width(ws5)

# === Sheet 6: Orders by BAN Summary ===
ws6 = wb.create_sheet("Orders by BAN")
headers6 = ['Current BAN Name', 'New BAN Name', 'Order Count', 'Total MRC']
ws6.append(headers6)
style_header(ws6, headers6, header_fill_blue)

ban_summary = orders_to_move_df.groupby(['current_ban_name', 'new_ban_name']).agg(
    order_count=('ordernumber', 'count'),
    total_mrc=('sof_mrc__c', 'sum')
).reset_index().sort_values('order_count', ascending=False)

for _, row in ban_summary.iterrows():
    ws6.append([
        row['current_ban_name'],
        row['new_ban_name'],
        row['order_count'],
        row['total_mrc']
    ])

ws6.freeze_panes = 'A2'
auto_width(ws6)

# Save
wb.save(output_file)
print(f"\n✅ Output saved to: {output_file}")

In [ ]:
# === Cleanup ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")